# Core 파이프라인 테스트

In [9]:
import os
import shutil
import uuid # core.py의 process_document 함수에서 사용될 수 있음
import google.generativeai as genai
from dotenv import load_dotenv # .env 파일 사용 시

# --- 모듈 임포트 ---
# 이전에 분리한 모듈들이 현재 주피터 노트북 파일과 같은 디렉토리, 
# 또는 파이썬 경로에 있다고 가정합니다.

# 1. 설정 로드 (config.py)
try:
    import config
    print("config.py 로드 성공.")
except ImportError:
    print("오류: config.py 파일을 찾을 수 없습니다. 설정을 로드할 수 없습니다.")
    # config.py가 없다면 필수 설정값들을 이 셀에서 직접 정의해야 합니다.
    # 예: GOOGLE_API_KEY = "YOUR_API_KEY" 등 (권장하지 않음)
    raise # 테스트 진행 불가

# 2. 프롬프트 템플릿 로드 (Prompt.py)
try:
    from Prompt import gemini_prompt as PROMPT_TEMPLATE
    print("Prompt.py 에서 PROMPT_TEMPLATE 로드 성공.")
except ImportError:
    print("오류: Prompt.py 또는 gemini_prompt 변수를 찾을 수 없습니다.")
    PROMPT_TEMPLATE = "OCR Data:\n{ocr_chunk_list_placeholder}\n\nImage Context:\n(이미지 있음)" # 임시
    print(f"임시 프롬프트 템플릿 사용: {PROMPT_TEMPLATE[:50]}...")


# 3. 핵심 처리 모듈 로드 (core.py)
try:
    from core import process_document
    print("core.py 에서 process_document 함수 로드 성공.")
except ImportError:
    print("오류: core.py 또는 process_document 함수를 찾을 수 없습니다.")
    raise

# 4. 시각화 모듈 로드 (visualizer.py)
try:
    from visualizer import draw_underlines_for_incorrect_answers_enhanced
    print("visualizer.py 에서 draw_underlines_for_incorrect_answers_enhanced 함수 로드 성공.")
except ImportError:
    print("경고: visualizer.py 또는 draw_underlines_for_incorrect_answers_enhanced 함수를 찾을 수 없습니다.")
    print("      시각화 테스트는 진행할 수 없습니다.")
    # 이 경우, 아래 시각화 셀 실행 시 오류가 발생하거나 건너뛰도록 처리해야 합니다.
    pass 

# --- API 키 설정 ---
load_dotenv() # .env 파일에서 GOOGLE_API_KEY 로드 시도
API_KEY_TO_USE = os.getenv("GOOGLE_API_KEY") if os.getenv("GOOGLE_API_KEY") else config.GOOGLE_API_KEY

if not API_KEY_TO_USE:
    raise ValueError("GOOGLE_API_KEY가 .env 또는 config.py를 통해 설정되지 않았습니다.")
genai.configure(api_key=API_KEY_TO_USE)
print("Google Generative AI SDK 설정 완료.")

print("\n기본 설정 및 모듈 임포트 완료.")

config.py 로드 성공.
Prompt.py 에서 PROMPT_TEMPLATE 로드 성공.
core.py 에서 process_document 함수 로드 성공.
visualizer.py 에서 draw_underlines_for_incorrect_answers_enhanced 함수 로드 성공.
Google Generative AI SDK 설정 완료.

기본 설정 및 모듈 임포트 완료.


In [10]:
# --- 처리할 문서 및 관련 경로 설정 ---

# 1. 처리할 입력 문서 경로 (사용자 환경에 맞게 수정)
#    config.py에 정의된 BASE_INPUT_DIR을 활용하거나, 직접 절대 경로를 지정할 수 있습니다.
# INPUT_DOCUMENT_PATH = os.path.join(config.BASE_INPUT_DIR, "예시_한국사.pdf") 
INPUT_DOCUMENT_PATH = '/Users/ki/Desktop/Google Drive/Dev/Ecode/OCR_Test/예시_한국사.pdf' # 사용자님 경로 예시

# 2. 서비스 계정 파일 경로 (config.py에서 가져옴)
SERVICE_ACCOUNT_FILE_PATH = config.SERVICE_ACCOUNT_FILE

# 3. 임시 폴더 기본 경로 (config.py에서 가져옴)
#    process_document 함수 내에서 이 기본 경로 하위에 고유한 폴더를 생성합니다.
TEMP_BASE_DIR_FOR_CORE = config.BASE_TEMP_DIR

# 4. LLM 설정 (config.py에서 가져옴)
GENERATION_CONFIG_FOR_CORE = config.GENERATION_CONFIG
SAFETY_SETTINGS_FOR_CORE = config.SAFETY_SETTINGS

# 입력 파일 존재 여부 확인
if not os.path.exists(INPUT_DOCUMENT_PATH):
    print(f"오류: 입력 파일 '{INPUT_DOCUMENT_PATH}'을 찾을 수 없습니다. 경로를 확인해주세요.")
    # 여기서 실행을 멈추거나, 다음 셀에서 오류를 발생시킬 수 있습니다.
    raise FileNotFoundError(f"입력 파일 없음: {INPUT_DOCUMENT_PATH}")

print(f"입력 문서: {INPUT_DOCUMENT_PATH}")
print("문서 처리 실행 준비 완료.")

입력 문서: /Users/ki/Desktop/Google Drive/Dev/Ecode/OCR_Test/예시_한국사.pdf
문서 처리 실행 준비 완료.


In [11]:
# --- [단계 1] 문서 처리 실행 ---
print(f"'{INPUT_DOCUMENT_PATH}' 문서에 대한 핵심 처리 시작...")

# core.py의 process_document 함수 호출
# 반환값: all_consolidated_data, all_rag_ready_data, image_paths, 생성된_임시폴더_경로
all_consolidated_data, all_rag_ready_data, image_paths, temp_folder_used_by_core = process_document(
    input_file_path=INPUT_DOCUMENT_PATH,
    service_account_file=SERVICE_ACCOUNT_FILE_PATH, # 셀 2에서 설정
    temp_base_dir=TEMP_BASE_DIR_FOR_CORE,          # 셀 2에서 설정
    prompt_template=PROMPT_TEMPLATE,               # 셀 1에서 로드
    generation_config=GENERATION_CONFIG_FOR_CORE,  # 셀 2에서 설정 (config.py 값)
    safety_settings=SAFETY_SETTINGS_FOR_CORE       # 셀 2에서 설정 (config.py 값)
)

print("\n--- 문서 처리 완료 (core.py) ---")
print(f"생성된 이미지 경로 개수: {len(image_paths)}")
if image_paths:
    print(f"  첫 번째 이미지 경로: {image_paths[0]}")
print(f"총 통합 데이터 항목 수 (all_consolidated_data): {len(all_consolidated_data)}")
print(f"총 RAG 준비 데이터 항목 수 (all_rag_ready_data): {len(all_rag_ready_data)}")

if all_rag_ready_data:
    print("\n--- RAG 준비 데이터 (all_rag_ready_data) 샘플 (상위 5개) ---")
    for i, item_list in enumerate(all_rag_ready_data[:5]):
        if item_list and len(item_list) > 1 and item_list[0] and isinstance(item_list[0], list) and item_list[1] and isinstance(item_list[1], list):
            id_list_str = f"ID({','.join(map(str, item_list[0]))})"
            text_str = item_list[1][0][:70] if item_list[1] else "" 
            print(f"  {id_list_str}, Text: '{text_str}...'")
        else:
            print(f"  Warning: Malformed RAG data item at index {i}: {item_list}")
else:
    print("\n생성된 RAG 준비 데이터가 없습니다.")

if temp_folder_used_by_core:
    print(f"\nPDF 변환 이미지가 저장된 임시 폴더: {temp_folder_used_by_core}")
    print("이 폴더는 시각화 테스트 후 또는 노트북 세션 종료 시 수동/자동으로 삭제해야 합니다.")

print("\n핵심 처리 단계 완료. `all_consolidated_data`, `all_rag_ready_data`, `image_paths`, `temp_folder_used_by_core` 변수가 생성되었습니다.")

'/Users/ki/Desktop/Google Drive/Dev/Ecode/OCR_Test/예시_한국사.pdf' 문서에 대한 핵심 처리 시작...
문서 처리 시작: /Users/ki/Desktop/Google Drive/Dev/Ecode/OCR_Test/예시_한국사.pdf
임시 폴더 '/Users/ki/PycharmProjects/Ecode_AI/Real/local_temp/pdf_pages_예시_한국사_662e31b4' 생성 완료.
PDF 파일 변환 중: '/Users/ki/Desktop/Google Drive/Dev/Ecode/OCR_Test/예시_한국사.pdf' (시간이 다소 소요될 수 있습니다)...
PDF 파일이 성공적으로 2개의 이미지로 변환되어 '/Users/ki/PycharmProjects/Ecode_AI/Real/local_temp/pdf_pages_예시_한국사_662e31b4'에 저장되었습니다.


[페이지 1/2 처리 시작] (예시_한국사_page_1.png)
  [페이지 1 - 단계 1] OCR 실행 중...
    페이지 1: 전체 OCR 및 ID 할당 완료 (총 513 청크).
    [페이지 1, 블록 0 - LLM 호출 준비] (청크 306개)
    페이지 1, 블록 0: LLM 호출 중 (모델: gemini-1.5-flash-latest)...
    [페이지 1, 블록 1 - LLM 호출 준비] (청크 207개)
    페이지 1, 블록 1: LLM 호출 중 (모델: gemini-1.5-flash-latest)...
  LLM Candidate Finish Reason: 1 (Value: 1)
    [페이지 1, 블록 1 - LLM 결과 처리 및 통합]...
    페이지 1, 블록 1 처리 완료. Consolidated: 207, RAG: 22
  LLM Candidate Finish Reason: 1 (Value: 1)
    [페이지 1, 블록 0 - LLM 결과 처리 및 통합]...
    페이지 1, 블록 0 처리 

In [12]:
all_rag_ready_data

[[[1, 0, 1, 1], ['경상도']],
 [[1, 0, 2, 1], ['대구']],
 [[1, 0, 3, 1], ['후삼국 통일 과정 중 공산 전투']],
 [[1, 0, 4, 1], ['대구 팔공산 부인사(초조 대장경 소실, 몽골 2차 침입)']],
 [[1, 0, 5, 1], ['국채 보상 운동(1907), 국채 보상 기성회']],
 [[1, 0, 6, 1], ['대한 회(1915), 공화정체 국가 건설 지향, 총사령 박상진, 군대식 조직']],
 [[1, 0, 7, 1], ['대구 2·28 민주화 운동(4·19 혁명 시발점), 이승만 정부 반발 민주화 운동']],
 [[1, 0, 8, 1], ['경주(서라벌, 사로국, 계림)']],
 [[1, 0, 9, 1], ['김유신 묘(12지 신상)']],
 [[1, 0, 10, 1], ['천마총(돌무지덧널무덤, 천마도)']],
 [[1, 0, 11, 1], ['황룡사(진흥왕 건립), 황룡사 9층 석탑(선덕여왕, 자장의 건의)']],
 [[1, 0, 12, 1], ['분황사 모전 석탑(신라 석탑 중 가장 오래된 석탑)']],
 [[1, 0, 13, 1], ['불국사(석가탑 - 무구정광대다라니경 발굴, 다보탑), 석굴암']],
 [[1, 0, 14, 1], ['안동']],
 [[1, 0, 15, 1], ['고창 전투']],
 [[1, 0, 16, 1], ['도산 서원(퇴계 이황), 병산 서원(서애 류성룡), 예안 향약(이왕)']],
 [[1, 0, 17, 1], ['법흥사지 7층 전탑(고려)']],
 [[1, 0, 18, 1], ['안동 이천동 석불(고려), 봉정사 극락전(고려, 주심포)']],
 [[1, 0, 19, 1], ['안동 놋다리밟기(공민왕, 노국대장공주 유래)']],
 [[1, 0, 20, 1], ['임청각(석주 이상룡: 임시정부 초대 국무령, 경학사 설립)']],
 [[1, 0, 21, 1], ['이육사 고향(저항 시인, 절정, 광야, 의열단)']],
 [[1, 0, 22, 1], ['영주']],

# 시각화 테스트

In [13]:
# --- [단계 2 준비] 시각화를 위한 오답 ID 설정 ---

# 이 리스트는 RAG 시스템의 결과라고 가정하며, 사용자가 직접 수정하여 테스트합니다.
# ID는 (page_num, block_id, y_idx, x_idx) 튜플 형태여야 하며,
# 위 셀에서 생성된 `all_rag_ready_data`에 실제로 존재하는 ID여야 합니다.
mock_incorrect_rag_ids = [
    # 예시: (1, 0, 3, 1), (2, 0, 6, 1) 
    # 아래는 이전 사용자님이 제공해주신 ID 목록입니다.
    # 실제 데이터에 맞게 확인 후 사용하거나, 위 all_rag_ready_data 샘플 출력을 보고 유효한 ID로 채워주세요.
    (1, 0, 3, 1), 
    (1, 0, 7, 1),
    (1, 1, 3, 1),
    (1, 1, 4, 1),
    # (1, 1, 17, 1), # 이 ID가 실제 생성된 all_rag_ready_data에 있는지 확인 후 사용
    (2, 0, 6, 1),
    (2,0,20,1)
]

# (선택 사항) 실제 all_rag_ready_data에 있는 ID인지 간단히 검증하는 로직
if 'all_rag_ready_data' in globals() and all_rag_ready_data:
    valid_mock_ids_for_viz = []
    rag_ids_set = {tuple(entry[0]) for entry in all_rag_ready_data if isinstance(entry, list) and len(entry) > 0 and isinstance(entry[0], list) and len(entry[0]) == 4}
    for mock_id in mock_incorrect_rag_ids:
        if mock_id in rag_ids_set:
            valid_mock_ids_for_viz.append(mock_id)
        else:
            print(f"경고: 목업 오답 ID {mock_id}가 생성된 RAG 데이터에 없습니다. 시각화에서 제외됩니다.")
    
    if not valid_mock_ids_for_viz:
        print("경고: 시각화할 유효한 목업 오답 ID가 없습니다.")
    else:
        print(f"시각화에 사용될 유효 오답 ID: {valid_mock_ids_for_viz}")
else:
    print("경고: `all_rag_ready_data`가 없어 오답 ID 유효성 검사를 할 수 없습니다. `mock_incorrect_rag_ids`를 그대로 사용합니다.")
    valid_mock_ids_for_viz = mock_incorrect_rag_ids


# 시각화 결과 저장 폴더 설정 (config.py의 BASE_OUTPUT_DIR 아래에 생성)
OUTPUT_VISUALIZATION_FOLDER = os.path.join(config.BASE_OUTPUT_DIR, "notebook_visualized_errors")

print(f"\n오답 ID 설정 완료: {valid_mock_ids_for_viz}")
print(f"시각화 결과 저장 폴더: {OUTPUT_VISUALIZATION_FOLDER}")

시각화에 사용될 유효 오답 ID: [(1, 0, 3, 1), (1, 0, 7, 1), (1, 1, 3, 1), (1, 1, 4, 1), (2, 0, 6, 1), (2, 0, 20, 1)]

오답 ID 설정 완료: [(1, 0, 3, 1), (1, 0, 7, 1), (1, 1, 3, 1), (1, 1, 4, 1), (2, 0, 6, 1), (2, 0, 20, 1)]
시각화 결과 저장 폴더: /Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors


In [14]:
# --- [단계 2 실행] 오답 시각화 ---

if 'draw_underlines_for_incorrect_answers_enhanced' in globals() and \
   'all_consolidated_data' in globals() and all_consolidated_data and \
   'image_paths' in globals() and image_paths and \
   valid_mock_ids_for_viz: # 유효한 오답 ID가 있을 때만 실행

    print(f"\n오답 시각화 함수 호출 (ID: {valid_mock_ids_for_viz})...")
    draw_underlines_for_incorrect_answers_enhanced(
        incorrect_rag_ids=valid_mock_ids_for_viz,
        all_consolidated_data=all_consolidated_data,
        all_rag_ready_data=all_rag_ready_data, 
        page_image_paths=image_paths,
        output_folder=OUTPUT_VISUALIZATION_FOLDER,
        # underline_color, underline_thickness, underline_offset 등은 기본값 사용
    )
    print(f"\n시각화 작업 완료. '{OUTPUT_VISUALIZATION_FOLDER}' 폴더를 확인하세요.")
else:
    if 'draw_underlines_for_incorrect_answers_enhanced' not in globals():
        print("시각화 함수(draw_underlines_for_incorrect_answers_enhanced)가 로드되지 않았습니다.")
    else:
        print("시각화에 필요한 데이터(all_consolidated_data, image_paths, 또는 유효한 오답 ID)가 준비되지 않았습니다.")


오답 시각화 함수 호출 (ID: [(1, 0, 3, 1), (1, 0, 7, 1), (1, 1, 3, 1), (1, 1, 4, 1), (2, 0, 6, 1), (2, 0, 20, 1)])...

--- '다음 ID 이전까지' 밑줄 그리기 작업 시작 (visualizer.py) ---
  출력 폴더 '/Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors' 생성 완료.
  페이지 1 밑줄 처리 완료. 저장 경로: /Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors/page_1_visualized_예시_한국사_page_1.png
  페이지 2 밑줄 처리 완료. 저장 경로: /Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors/page_2_visualized_예시_한국사_page_2.png

총 2개의 이미지에 밑줄 작업 완료. '/Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors' 폴더를 확인하세요.
--- '다음 ID 이전까지' 밑줄 그리기 작업 종료 ---

시각화 작업 완료. '/Users/ki/PycharmProjects/Ecode_AI/Real/local_outputs/notebook_visualized_errors' 폴더를 확인하세요.


In [15]:
# --- [단계 3] 임시 폴더 정리 ---

# process_document 함수가 반환한 임시 폴더 경로 (temp_folder_used_by_core)를 사용
if 'temp_folder_used_by_core' in globals() and temp_folder_used_by_core and os.path.exists(temp_folder_used_by_core):
    try:
        shutil.rmtree(temp_folder_used_by_core)
        print(f"\n임시 PDF 변환 폴더 '{temp_folder_used_by_core}' 삭제 완료.")
    except Exception as e:
        print(f"\n임시 PDF 변환 폴더 '{temp_folder_used_by_core}' 삭제 중 오류: {e}")
else:
    if 'temp_folder_used_by_core' in globals() and temp_folder_used_by_core:
        print(f"\n임시 PDF 변환 폴더 '{temp_folder_used_by_core}'가 존재하지 않거나 이미 삭제되었습니다.")
    else:
        print("\n정리할 임시 PDF 변환 폴더 정보가 없습니다.")


임시 PDF 변환 폴더 '/Users/ki/PycharmProjects/Ecode_AI/Real/local_temp/pdf_pages_예시_한국사_662e31b4' 삭제 완료.
